### Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

### Data Prep

In [4]:
ddf = dd.read_parquet('./data/parquet/data-*.parquet')

In [5]:
#check column names
ddf.columns

Index(['index', 'Invoice/Item Number', 'Date', 'Store Number', 'Store Name',
       'Address', 'City', 'Zip Code', 'Store Location', 'County Number',
       'County', 'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'],
      dtype='object')

In [6]:
#check size of file for correct number of rows
ddf.shape[0].compute()

23346088

In [247]:
#split main df into smaller subcategory df's
county_info = ['County Number', 'County']
vendor_info = ['Vendor Name', 'Vendor Number']
product_info = ['Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)']
price_info = ['Item Number', 'State Bottle Cost', 'State Bottle Retail', 'Date']
store_info = ['Store Number', 'Store Name', 'City', 'Zip Code', 'County Number', 'Date']

### Counties Sub-Frame

In [8]:
ddf = ddf[county_info]

In [9]:
#get rid of duplicate county rows
ddf = ddf.drop_duplicates().compute()

In [10]:
ddf

,County Number,County
0,68.0,Monroe
1,17.0,Cerro Gordo
2,1.0,Adair
3,85.0,Story
4,21.0,Clay
...,...,...
29060,26.0,DAVIS
34735,13.0,CALHOUN
40134,93.0,WAYNE
12288,2.0,ADAMS


In [11]:
#check for NA county numbers
ddf[ddf['County Number'].isna()]

,County Number,County
2052,NaN,None
5977,NaN,EL PASO


In [12]:
#drop rows w/ NA for county number
ddf = ddf[~ddf['County Number'].isna()]

#add in row for unknown
ddf = ddf.append(pd.Series({'County Number': 999, 'County': 'Unknown'}), ignore_index = True)
ddf

,County Number,County
0,68.0,Monroe
1,17.0,Cerro Gordo
2,1.0,Adair
3,85.0,Story
4,21.0,Clay
...,...,...
196,26.0,DAVIS
197,13.0,CALHOUN
198,93.0,WAYNE
199,2.0,ADAMS


In [17]:
#gets rid of duplicates from typos, mispelling, etc
ddf = ddf.loc[ddf['County Number'].drop_duplicates().index]

In [18]:
#contains all 99 Iowa counties, plus row for unknown
ddf

,County Number,County
0,68.0,Monroe
1,17.0,Cerro Gordo
2,1.0,Adair
3,85.0,Story
4,21.0,Clay
...,...,...
95,26.0,Davis
96,36.0,Fremont
97,93.0,Wayne
98,2.0,Adams


In [19]:
#convert county numbers to integer type
ddf.loc[:, 'County Number'] = ddf['County Number'].astype(int)

In [20]:
#rename columns
ddf.columns = ['CountyNumber', 'County']

In [21]:
#export df as csv
ddf.to_csv('./data/county.csv')

### Vendors Sub-Frame

In [132]:
#reread in parquet files
ddf = dd.read_parquet('./data/parquet/data-*.parquet')

In [133]:
#select vendor info columns
ddf = ddf[vendor_info]

In [134]:
#drop duplicate vendor rows
ddf = ddf.drop_duplicates().compute()

In [135]:
#rename columns
ddf.columns = ['VendorName', 'VendorNumber']

In [136]:
#prepping to check for NA values in columns
idx_na = ddf['VendorNumber'].isna()
name_na = ddf['VendorName'].isna()

In [137]:
#checks for NA vendor number
ddf[idx_na]

,VendorName,VendorNumber
37814,None,NaN
740,Reservoir Distillery,NaN


In [138]:
#checks for NA vendor name
ddf[name_na]

,VendorName,VendorNumber
37814,None,NaN


In [139]:
ddf

,VendorName,VendorNumber
0,Luxco-St Louis,434.0
1,"WILLIAM GRANT AND SONS, INC.",240.0
3,Jim Beam Brands,65.0
4,"Sazerac Co., Inc.",421.0
5,Phillips Beverage Company,380.0
...,...,...
35836,Leatherbee Distillers,252.0
6471,Breuckelen Distilling,119.0
37654,Sovereign Brands,482.0
29558,"Phenix Brands, LLC",386.0


In [140]:
#removes NA vendor name row
ddf = ddf[~ddf['VendorNumber'].isna()]

In [141]:
ddf

,VendorName,VendorNumber
0,Luxco-St Louis,434.0
1,"WILLIAM GRANT AND SONS, INC.",240.0
3,Jim Beam Brands,65.0
4,"Sazerac Co., Inc.",421.0
5,Phillips Beverage Company,380.0
...,...,...
35836,Leatherbee Distillers,252.0
6471,Breuckelen Distilling,119.0
37654,Sovereign Brands,482.0
29558,"Phenix Brands, LLC",386.0


In [142]:
#checking for repeat names
ddf['VendorName'].value_counts()

Casa 1921 LLC                        2
Levecke Corporation                  2
Luxco-St Louis                       1
Old Elk Distilleries LLC             1
HPSepicurean LLC / Preiss Imports    1
                                    ..
The Country Vintner                  1
Brown Forman Corp.                   1
DIAGEO AMERICAS                      1
Iconic Brands, Inc.                  1
Tanteo Tequila                       1
Name: VendorName, Length: 552, dtype: int64

In [143]:
#check for repeat vendor numbers
ddf['VendorNumber'].value_counts()

192.0    4
391.0    4
803.0    4
214.0    4
338.0    3
        ..
216.0    1
269.0    1
583.0    1
548.0    1
462.0    1
Name: VendorNumber, Length: 400, dtype: int64

In [144]:
#removes duplicate vendor numbers
ddf = ddf.drop_duplicates(subset = ['VendorNumber'])

In [145]:
ddf

,VendorName,VendorNumber
0,Luxco-St Louis,434.0
1,"WILLIAM GRANT AND SONS, INC.",240.0
3,Jim Beam Brands,65.0
4,"Sazerac Co., Inc.",421.0
5,Phillips Beverage Company,380.0
...,...,...
16631,Journeyman Distillery LLC,191.0
35836,Leatherbee Distillers,252.0
6471,Breuckelen Distilling,119.0
29558,"Phenix Brands, LLC",386.0


In [146]:
#checks that all value counts are now 1
ddf['VendorNumber'].value_counts(dropna = False)

434.0    1
607.0    1
617.0    1
602.0    1
554.0    1
        ..
465.0    1
209.0    1
118.0    1
108.0    1
462.0    1
Name: VendorNumber, Length: 400, dtype: int64

In [147]:
#add in row for unknown
ddf = ddf.append(pd.Series({'VendorNumber': 9999, 'VendorName': 'Unknown'}), ignore_index = True)

In [148]:
ddf

,VendorName,VendorNumber
0,Luxco-St Louis,434.0
1,"WILLIAM GRANT AND SONS, INC.",240.0
2,Jim Beam Brands,65.0
3,"Sazerac Co., Inc.",421.0
4,Phillips Beverage Company,380.0
...,...,...
396,Leatherbee Distillers,252.0
397,Breuckelen Distilling,119.0
398,"Phenix Brands, LLC",386.0
399,Tanteo Tequila,462.0


In [149]:
#checks that vendor numbers are integers
ddf['VendorNumber'].describe()

count     401.000000
mean      436.802993
std       521.376643
min        10.000000
25%       252.000000
50%       410.000000
75%       566.000000
max      9999.000000
Name: VendorNumber, dtype: float64

In [150]:
#export df as csv
ddf.to_csv('./data/vendor.csv')

### Products Sub-Frame

In [151]:
#reread in parquet files
ddf = dd.read_parquet('./data/parquet/data-*.parquet')

In [152]:
#select product info columns
ddf = ddf[product_info]

In [154]:
#drop duplicate product rows
ddf = ddf.drop_duplicates().compute()

In [155]:
ddf

,Item Number,Item Description,Pack,Bottle Volume (ml)
0,36308,Hawkeye Vodka,6,1750
1,45888,Sailor Jerry Spiced Navy Rum,6,1750
2,55084,Paramount Blackberry Brandy,24,375
3,67557,Kamora Coffee Liqueur,12,1000
4,77487,Tortilla Gold Dss,12,1000
...,...,...,...,...
26959,901036,Apricot Grappa,1,1000
34073,973579,Crave Chocolate Truffle,12,750
11453,902750,Margaritaville Oro,12,1000
23449,967220,Villa Massa Limoncello,6,750


In [156]:
#rename columns
ddf.columns = ['ItemNumber', 'ItemDescription', 'Pack', 'BottleVolume(ml)']

In [157]:
#prepping to check for NA values in columns
idx_na = ddf['ItemNumber'].isna()
desc_na = ddf['ItemDescription'].isna()
pack_na = ddf['Pack'].isna()
vol_na = ddf['BottleVolume(ml)'].isna()

In [158]:
#checks for NA item number
ddf[idx_na]

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)


In [159]:
#checks for NA item description
ddf[desc_na]

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)


In [160]:
#checks for NA pack
ddf[pack_na]

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)


In [161]:
#checks for NA bottle volume
ddf[vol_na]

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)


In [162]:
#check for repeat item numbers
ddf['ItemNumber'].value_counts()

965266    7
984160    6
16906     6
22219     6
984393    5
         ..
982785    1
35221     1
904006    1
902749    1
904010    1
Name: ItemNumber, Length: 11013, dtype: int64

In [169]:
#removes duplicate item numbers
ddf = ddf.drop_duplicates(subset = ['ItemNumber'])

In [171]:
#checks that all value counts are now 1
ddf['ItemNumber'].value_counts(dropna = False)

36308     1
100246    1
78059     1
989502    1
100812    1
         ..
904118    1
902968    1
936020    1
900002    1
904010    1
Name: ItemNumber, Length: 11013, dtype: int64

In [172]:
ddf

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)
0,36308,Hawkeye Vodka,6,1750
1,45888,Sailor Jerry Spiced Navy Rum,6,1750
2,55084,Paramount Blackberry Brandy,24,375
3,67557,Kamora Coffee Liqueur,12,1000
4,77487,Tortilla Gold Dss,12,1000
...,...,...,...,...
26959,901036,Apricot Grappa,1,1000
34073,973579,Crave Chocolate Truffle,12,750
11453,902750,Margaritaville Oro,12,1000
23449,967220,Villa Massa Limoncello,6,750


In [176]:
#identifies row with letter in item number
ddf[ddf['ItemNumber'] == 'x904631']

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)
16474,x904631,Tanqueray Gin Mini - Use 904631 Code,12,500


In [180]:
#removes letter from item number
ddf = ddf.replace('x904631', '904631')

In [181]:
#checks to see if all value counts are still 1
ddf['ItemNumber'].value_counts(dropna = False)

904631    2
36308     1
972427    1
26673     1
78059     1
         ..
554       1
904118    1
902968    1
936020    1
904010    1
Name: ItemNumber, Length: 11012, dtype: int64

In [183]:
#removes duplicate item numbers
ddf = ddf.drop_duplicates(subset = ['ItemNumber'])

In [184]:
#convert cols to integer type
ddf.loc[:, 'ItemNumber'] = ddf['ItemNumber'].astype(int)
ddf.loc[:, 'Pack'] = ddf['Pack'].astype(int)
ddf.loc[:, 'BottleVolume(ml)'] = ddf['BottleVolume(ml)'].astype(int)

In [188]:
#add in row for unknown
ddf = ddf.append(pd.Series({'ItemNumber': 99999999999, 'ItemDescription': 'Unknown', 'Pack': 0, 'BottleVolume(ml)': 0}), ignore_index = True)

In [189]:
ddf

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)
0,36308,Hawkeye Vodka,6,1750
1,45888,Sailor Jerry Spiced Navy Rum,6,1750
2,55084,Paramount Blackberry Brandy,24,375
3,67557,Kamora Coffee Liqueur,12,1000
4,77487,Tortilla Gold Dss,12,1000
...,...,...,...,...
11008,973579,Crave Chocolate Truffle,12,750
11009,902750,Margaritaville Oro,12,1000
11010,967220,Villa Massa Limoncello,6,750
11011,904010,Tanteo Jalapeno Tequila( do not use),6,750


In [190]:
#export df as csv
ddf.to_csv('./data/product.csv')

### Prices Sub-Frame

In [218]:
#reread in parquet files
ddf = dd.read_parquet('./data/parquet/data-*.parquet')

In [219]:
#select price info columns
ddf = ddf[price_info]

In [220]:
#drop duplicate price rows
ddf = ddf.drop_duplicates().compute()

In [221]:
ddf

,Item Number,State Bottle Cost,State Bottle Retail,Date
0,36308,7.13,10.70,06/06/2012
1,45888,18.67,28.01,12/02/2013
2,55084,3.55,5.33,04/06/2015
3,67557,8.39,12.59,10/22/2014
4,77487,4.51,6.76,07/30/2013
...,...,...,...,...
41042,43451,7.50,11.25,07/24/2013
41092,15677,16.49,24.74,08/26/2015
41103,73500,12.07,18.11,07/31/2014
41111,89099,23.48,35.22,03/19/2014


In [222]:
#rename columns
ddf.columns = ['ItemNumber', 'StateBottleCost', 'StateBottleRetail', 'Date']

In [223]:
#removes letter from item number
ddf = ddf.replace('x904631', '904631')

In [224]:
#convert item numbers to integer type
ddf.loc[:, 'ItemNumber'] = ddf['ItemNumber'].astype(int)

In [225]:
#convert date column to date type
ddf['Date'] = pd.to_datetime(ddf['Date'], infer_datetime_format=True)

In [226]:
ddf

,ItemNumber,StateBottleCost,StateBottleRetail,Date
0,36308,7.13,10.70,2012-06-06
1,45888,18.67,28.01,2013-12-02
2,55084,3.55,5.33,2015-04-06
3,67557,8.39,12.59,2014-10-22
4,77487,4.51,6.76,2013-07-30
...,...,...,...,...
41042,43451,7.50,11.25,2013-07-24
41092,15677,16.49,24.74,2015-08-26
41103,73500,12.07,18.11,2014-07-31
41111,89099,23.48,35.22,2014-03-19


In [230]:
#sory by date
ddf = ddf.sort_values(by='Date')

In [235]:
#filter out data before 2018
ddf = ddf[ddf['Date'] >= '2018-01-01']

In [236]:
ddf

,ItemNumber,StateBottleCost,StateBottleRetail,Date
28783,42723,7.49,11.24,2018-01-02
30030,54448,11.29,16.94,2018-01-02
26935,22151,12.45,18.68,2018-01-02
28770,34546,7.00,10.50,2018-01-02
26937,43316,11.99,17.99,2018-01-02
...,...,...,...,...
23386,89386,7.50,11.25,2022-02-28
30482,84166,7.50,11.25,2022-02-28
30466,34454,5.99,8.99,2022-02-28
23391,73054,25.00,37.50,2022-02-28


In [237]:
#prepping to check for NA values in columns
idx_na = ddf['ItemNumber'].isna()
cost_na = ddf['StateBottleCost'].isna()
ret_na = ddf['StateBottleRetail'].isna()
date_na = ddf['Date'].isna()

In [238]:
#checks for NA item number
ddf[idx_na]

,ItemNumber,StateBottleCost,StateBottleRetail,Date


In [239]:
#checks for NA bottle cost
ddf[cost_na]

,ItemNumber,StateBottleCost,StateBottleRetail,Date


In [240]:
#checks for NA bottle retail
ddf[ret_na]

,ItemNumber,StateBottleCost,StateBottleRetail,Date


In [241]:
#checks for NA date
ddf[date_na]

,ItemNumber,StateBottleCost,StateBottleRetail,Date


In [242]:
#export df as csv
ddf.to_csv('./data/price.csv')

### Stores Sub-Frame

In [250]:
#reread in parquet files
ddf = dd.read_parquet('./data/parquet/data-*.parquet')

In [251]:
#select store info columns
ddf = ddf[store_info]

In [252]:
#drop duplicate price rows
ddf = ddf.drop_duplicates().compute()

In [253]:
ddf

,Store Number,Store Name,City,Zip Code,County Number,Date
0,4011,Jim and Charlies Affiliated,ALBIA,52531,68.0,06/06/2012
1,2515,Hy-Vee Food Store #1 / Mason City,MASON CITY,50401,17.0,12/02/2013
2,3461,Hometown Foods / Stuart,STUART,50250,1.0,04/06/2015
3,2500,Hy-Vee Food Store #1 / Ames,AMES,50010,85.0,10/22/2014
4,2565,Hy-Vee Food Store / Spencer,SPENCER,51301,21.0,07/30/2013
...,...,...,...,...,...,...
2051,2478,Prairie Meadows,ALTOONA,50009,77.0,02/13/2012
40676,4338,Randhawa's Travel Center,MELBOURNE,50162,64.0,11/17/2012
17487,2506,Hy-Vee #1044 / Burlington,BURLINGTON,52601,29.0,10/30/2013
20268,2619,Hy-Vee Wine and Spirits / WDM,WEST DES MOINES,50266,77.0,05/07/2015


In [254]:
#rename columns
ddf.columns = ['StoreNumber', 'StoreName', 'City', 'ZipCode', 'CountyNumber', 'Date']

In [262]:
#prepping to check for NA values in columns
idx_na = ddf['StoreNumber'].isna()
name_na = ddf['StoreName'].isna()
city_na = ddf['City'].isna()
zip_na = ddf['ZipCode'].isna()
county_na = ddf['CountyNumber'].isna()
date_na = ddf['Date'].isna()

In [263]:
#checks for NA store number
ddf[idx_na]

,StoreNumber,StoreName,City,ZipCode,CountyNumber,Date


In [264]:
#checks for NA store name
ddf[name_na]

,StoreNumber,StoreName,City,ZipCode,CountyNumber,Date


In [265]:
#checks for NA city
ddf[city_na]

,StoreNumber,StoreName,City,ZipCode,CountyNumber,Date
22762,5140,A to Z Liquor,None,None,NaN,04/15/2019
29439,5140,A to Z Liquor,None,None,NaN,04/09/2019
39417,5140,A to Z Liquor,None,None,NaN,04/02/2019
40799,5251,Northside Liquor,None,None,NaN,04/04/2019
18710,2539,Hy-Vee Food Store / Iowa Falls,None,None,NaN,04/01/2019
...,...,...,...,...,...,...
40013,4725,Casey's General Store #1548 / Ankeny,None,None,NaN,09/22/2016
40608,5279,Casey's General Store # 2560,None,None,NaN,09/28/2016
29652,3973,Mmdg Spirits / Ames,None,None,NaN,09/30/2016
16605,4777,Casey's General Store #1365 / Paullina,None,None,NaN,11/28/2017


In [266]:
#checks for NA zip code
ddf[zip_na]

,StoreNumber,StoreName,City,ZipCode,CountyNumber,Date
22762,5140,A to Z Liquor,None,None,NaN,04/15/2019
29439,5140,A to Z Liquor,None,None,NaN,04/09/2019
39417,5140,A to Z Liquor,None,None,NaN,04/02/2019
40799,5251,Northside Liquor,None,None,NaN,04/04/2019
18710,2539,Hy-Vee Food Store / Iowa Falls,None,None,NaN,04/01/2019
...,...,...,...,...,...,...
40013,4725,Casey's General Store #1548 / Ankeny,None,None,NaN,09/22/2016
40608,5279,Casey's General Store # 2560,None,None,NaN,09/28/2016
29652,3973,Mmdg Spirits / Ames,None,None,NaN,09/30/2016
16605,4777,Casey's General Store #1365 / Paullina,None,None,NaN,11/28/2017


In [267]:
#checks for NA county number
ddf[county_na]

,StoreNumber,StoreName,City,ZipCode,CountyNumber,Date
2052,4247,Fareway Stores #879 / Belmond,BELMOND,50421,NaN,01/20/2015
2951,3782,Bender Foods / Guttenberg,GUTTENBERG,52052,NaN,07/16/2012
4260,4247,Fareway Stores #879 / Belmond,BELMOND,50421,NaN,09/08/2015
4300,4247,Fareway Stores #879 / Belmond,BELMOND,50421,NaN,11/10/2015
5372,3782,Bender Foods / Guttenberg,GUTTENBERG,52052,NaN,10/14/2013
...,...,...,...,...,...,...
40013,4725,Casey's General Store #1548 / Ankeny,None,None,NaN,09/22/2016
40608,5279,Casey's General Store # 2560,None,None,NaN,09/28/2016
29652,3973,Mmdg Spirits / Ames,None,None,NaN,09/30/2016
16605,4777,Casey's General Store #1365 / Paullina,None,None,NaN,11/28/2017


In [268]:
#checks for NA date
ddf[date_na]

,StoreNumber,StoreName,City,ZipCode,CountyNumber,Date


In [256]:
#convert store numbers to integer type
ddf.loc[:, 'StoreNumber'] = ddf['StoreNumber'].astype(int)

In [270]:
#convert county numbers to integer type and fills na's as 9999
ddf.loc[:, 'CountyNumber'] = ddf['CountyNumber'].fillna(9999).astype(int)

In [271]:
#convert date column to date type
ddf['Date'] = pd.to_datetime(ddf['Date'], infer_datetime_format=True)

In [272]:
#sory by date
ddf = ddf.sort_values(by='Date')

In [274]:
#filter out data before 2018
ddf = ddf[ddf['Date'] >= '2018-01-01']

In [275]:
ddf

,StoreNumber,StoreName,City,ZipCode,CountyNumber,Date
24166,5224,Keystone Liquor & Wine / Coralville,Coralville,52241,52,2018-01-02
24130,4872,Casey's General Store #2644 / Earlham,Earlham,50072,61,2018-01-02
21828,5254,Discount Liquor,Cedar Rapids,52402,57,2018-01-02
23760,4485,DYNO'S 51 / SANBORN,Sanborn,51248,71,2018-01-02
24246,4900,Kwik Stop 4 / Waterloo,Waterloo,50703,7,2018-01-02
...,...,...,...,...,...,...
22030,4379,Kum & Go #2091 / Ashworth / WDM,West Des Moines,50266,77,2022-02-28
22131,5909,EZ Stop 1 / Dubuque,Dubuque,52001,31,2022-02-28
22176,6046,Casey's General Store #2150 / WDM,West Des Moines,50265,77,2022-02-28
21167,6240,Raceway 80 / Newton,Newton,50208,50,2022-02-28


In [276]:
#export df as csv
ddf.to_csv('./data/store.csv')